In [1]:
import nbimporter
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from hyperopt import hp, fmin, STATUS_OK, tpe, Trials
from hyperopt.pyll.base import scope

from Preprop import *
from Feature_En import *

In [2]:
d_path = 'datas'
df = data_preprocess(d_path, use_test=False)


D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x4 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [3]:
df = generate_features(df)

Feature_En.ipynb:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "            dataframes_profile.loc[:, item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')\n",
Feature_En.ipynb:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        for featur in category_vars:\n",
Feature_En.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [4]:
df = df.loc[:, df.columns[3:]]
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], 
                                                    test_size=0.2, random_state=0, shuffle=False)

In [15]:
def objective_function(params):
    model = CatBoostClassifier(n_estimators=50, random_state=0, **params)
    model.fit(X_train, y_train)
    
    # prediction_proba - т.е. вероятностная характеристика
    # y_pred = model.predict_proba(X_test)[:, 1]
    # y_pred = y_pred.reshape(1, -1)
    
    # обычный prediction
    y_pred =  model.predict(X_test)
    y_pred = y_pred.reshape(1, -1)
    
    metric = metrics.ndcg_score(y_test.values.reshape(1, -1), y_pred, k=20)
    # metric2 = metrics.ndcg_score(y_test.values.reshape(1, -1), y_pred2, k=20)
    
    return {'loss': -metric, 'status': STATUS_OK}

In [16]:
space = { 'depth': scope.int(hp.quniform('depth', 4, 10, 1)),
          'max_bin': scope.int(hp.quniform('max_bin', 5, 50, 1)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
          'random_strength': hp.loguniform('random_strength', np.log(1e-9), np.log(10)),
          'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
          'loss_function': hp.choice('loss_function', ['Logloss', 'CrossEntropy'])}

In [17]:
trials = Trials()
best_param = fmin(objective_function, space, algo=tpe.suggest, max_evals=10, 
                  trials=trials, rstate=np.random.default_rng(42))

0:	learn: 0.6568773	total: 681ms	remaining: 33.4s                                                                                                                                      

1:	learn: 0.6389309	total: 1.32s	remaining: 31.7s                                                                                                                                      

2:	learn: 0.6300163	total: 2.04s	remaining: 31.9s                                                                                                                                      

3:	learn: 0.6240329	total: 2.84s	remaining: 32.7s                                                                                                                                      

4:	learn: 0.6199224	total: 3.52s	remaining: 31.7s                                                                                                                                      

5:	learn: 0.6170527	total: 4.08s	remaining: 29.9s                          

In [18]:
print(best_param)

{'depth': 10.0, 'l2_leaf_reg': 5.555897323622429, 'learning_rate': 0.4359843989797999, 'loss_function': 0, 'max_bin': 50.0, 'random_strength': 0.009790073389282446}


Лучшие параметры оказались следующие (при loss = -0,5933975071773377):
'depth': 10.0, 
'l2_leaf_reg': 5.555897323622429, 
'learning_rate': 0.4359843989797999, 
'loss_function': 'Logloss', 
'max_bin': 50.0, 
'random_strength': 0.009790073389282446

Перенесём их в Fit.ipynb
